In [74]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier 
from sklearn.cross_validation import KFold, cross_val_score,train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import confusion_matrix
import matplotlib as mpl
mpl.rc('xtick', labelsize=25) 
mpl.rc('ytick', labelsize=25)
import itertools

from sklearn.externals import joblib
from collections import Counter
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
#特征选择 
%matplotlib inline

In [96]:
# 导入数据
train = pd.read_pickle("/home/zhh/文档/阿里巴巴/data/6.合表/训练集总表08131042.pickle")
train.columns
X = train.drop(["标签"],axis = 1)
y = train["标签"].map(lambda x:-1 if x==1 else 1)

In [97]:
#*************y的骗保和不偏保的比例
y.value_counts()/y.shape[0]

 1    0.950564
-1    0.049436
Name: 标签, dtype: float64

In [98]:
#*******************数据进行归一化******************************
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)
scaler = StandardScaler()
scaler.fit(X_train)
new_X_train = scaler.fit_transform(X_train)
new_X_test = scaler.transform(X_test)

In [101]:
nu = 0.04
model = svm.OneClassSVM(nu=nu,kernel='rbf', gamma=0.00005) 
model.fit(new_X_train)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=5e-05, kernel='rbf',
      max_iter=-1, nu=0.04, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [102]:

from sklearn import metrics  
preds = model.predict(new_X_test)  
targs = y_test
print(classification_report(targs,preds))

             precision    recall  f1-score   support

         -1       0.29      0.19      0.23       150
          1       0.96      0.98      0.97      3042

avg / total       0.93      0.94      0.93      3192



 # part 孤立森林

In [103]:
from sklearn.ensemble import IsolationForest

In [112]:
# 导入数据
train = pd.read_pickle("/home/zhh/文档/阿里巴巴/data/6.合表/训练集总表08131042.pickle")
train.columns
X = train.drop(["标签"],axis = 1)
y = train["标签"].map(lambda x:-1 if x==1 else 1)

In [129]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3)
scaler = StandardScaler()
scaler.fit(X_train)
new_X_train = scaler.fit_transform(X_train)
new_X_test = scaler.transform(X_test)

In [136]:
itree = IsolationForest(bootstrap=True,n_estimators=1500,max_features=.3)
itree.fit(new_X_train)

IsolationForest(bootstrap=True, contamination=0.1, max_features=0.3,
        max_samples='auto', n_estimators=1500, n_jobs=1, random_state=None,
        verbose=0)

In [137]:
preds = itree.predict(new_X_test)
print(classification_report(y_test,preds))

             precision    recall  f1-score   support

         -1       0.17      0.39      0.24       217
          1       0.97      0.91      0.94      4571

avg / total       0.93      0.89      0.91      4788

